# Introduction

This notebook implements a gradient-boosted decision tree ensemble via XGBoost. The model uses 63 input features to predict the single binary outcome of whether or not a particular loan is bad. 1771 models are built with unique hyperparameters to find the model structure that maximizes performance on key target metrics. The highest performing model correctly identifies whether a loan will be good or bad in 6% of validation set cases. 

Data is split into three sets: train, validation, and test. Individual models are fitted on the training data. Performance metrics are reported on validation data, which is not used in training. Once a model has been chosen, test data will provide an estimate of its performance in future use. Cross-validation is used to generate five unique splits of training and validation data for use in fitting, providing performance estimates less biased by the individual dataset. 

Six hyperparameters are tuned in order to optimize performance of the model (tested values in parentheses):
* Maximum Tree Depth: How many levels of splits can an individual tree contain? (2, 4, 8, 16, 32)
* Minimum Child Weight: What is the minimum sample size of a new node/leaf? (1, 4, 8, 12, 16, 20, 24, 48)
* Subsample Fraction: What fraction of the training data is each tree trained on? (0.70, 1.00)
* Lambda: Weight multiplier on the L2 regularization parameter. (0, 0.5, 1)
* Gamma: Prune nodes below the given threshold of performance gain. (0, 0.5, 1, 5, 10, 50, 100)
* Eta: Reduce the weight of each successive tree by the given multiplier. (0.001, 0.01, 0.05, 0.1, 0.25, 0.5)

Six model performance metrics are tracked for analysis in the next notebook:
* Brier Score
* AUC-ROC Curve
    * TODO: Plot this
* Precision
* Recall
* Accuracy
* Balanced Accuracy

Future work on this question would include the specification of a custom scoring mechanism for fair lending metrics. By deriving each customer's demographic data from their application information, we could measure the default probabilities assigned to customers of different protected groups. We could compare their overall probabilities of default, as well as probabilities of false positive and false negative, in order to estimate the fair lending impact of our model.


Paper on XGBoost for Default Prediction: https://www.terry.uga.edu/sites/default/files/inline-files/Albanesi_Domossy_2021.pdf

In [7]:
import numpy as np
import pandas as pd
import scipy.stats
import xgboost
import tqdm
import pickle

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import RocCurveDisplay
from xgboost import XGBClassifier, DMatrix
import multiprocessing

from sklearn.model_selection import train_test_split

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 50)

In [8]:
# Import data and data dictionary
data = pd.read_pickle('output_data/02_data.pkl')

data_dict = pd.read_pickle('output_data/02_data_dict.pkl')

## Train, Test, and Validation Data

TODO: Fix description. I'm going to start with a train-validation-test split of 60/20/20. This is weighted more towards test data and less towards training data than others splits that are commonly used. This decision is motivated by concern of overfitting with a powerful model predicting such a small sample size. With a sizeable test set, we should have a clear understanding of how well the model performs on the sample. 

In [9]:
# Designate feature set

feature_set = {
    'all': data_dict.loc[data_dict['potential_feature']==True, 
                               'variable'].values,
    'no_credit': data_dict.loc[data_dict['eda_category'].isin(('personal_finance', 'other_info')),
                               'variable'].values,
    'significant': data_dict.loc[data_dict['pearson_p'] < 0.05, 
                               'variable'].values
    'significant_no_credit': data_dict.loc[(data_dict['pearson_p'] < 0.05) &
        (data_dict['eda_category'].isin(('personal_finance', 'other_info'))),
                                                          'variable'].values,
}


In [10]:
# Split into training, validation, and testing data
# Split chronologically to resolve the look-ahead problem

# Generate a new column to track the observation's model dataset category
data['model_set'] = 'train'

# Test Data comes from March 2011
data.loc[data['application_month_of_cycle'] == 6, 'model_set'] = 'test'

# Display results
print("Chronological method achieves Train-Val-Test split of approximately 85-15 proportions.")
print("Validation sets consisting of 20% of training data will be dynamically generated later.")
data['model_set'].value_counts()

Chronological method achieves Train-Val-Test split of approximately 85-15 proportions.
Validation sets consisting of 20% of training data will be dynamically generated later.


train    555
test      95
Name: model_set, dtype: int64

TODO: PowerPoint chart to explain where we divided the train, test, and validation data. 

In [11]:
# Split into x and y data
# Use the list of potential features from the data dictionary
X_train = data.loc[data['model_set'] == 'train']
X_test = data.loc[data['model_set'] == 'test']

# Designate Y data for train, validation, and test
y_train = data.loc[data['model_set'] == 'train', 'bad']
y_test = data.loc[data['model_set'] == 'test', 'bad']

## Sample Run

In [6]:
# Configure constant model settings
settings = {
    'booster': ['gbtree'], 
    'objective': ['binary:logistic'],
    'eval_metric': ['logloss'], # TODO: Suppress the warning down there. 
    'nthread': [multiprocessing.cpu_count()]
}

# Configure hyperparameters
# Currently building 1728 models
hypers = {
    'eta': [0.01, .10, .25, .50], # learning rate, reduces size of error adjustment
    'gamma': [0, 0.5, 5], # Raise to reduce overfitting (prevents low-magnitude leaves)    
    'min_child_weight': [1, 4, 16], # Raise to eliminate leaves with low sample weight
    'subsample': [.7, 1], # Each boosting round trains on X% of the training data
    'max_depth': [2, 4, 8], # Maximum depth of a single tree
    'reg_lambda': [0, 1], # L2 regularization
}

single_hyper = {k: v[0] for k, v in hypers.items()}

# Monitor hypers chosen by XGB
monitor_hypers = {
    'num_pbuffer': None,
    'num_feature': None
}

In [7]:
simple_dict = settings.copy()
new_simple_dict = {
    'eta': [.10], # learning rate, reduces size of error adjustment
    'gamma': [0], # Raise to reduce overfitting (prevents low-magnitude leaves)    
    'min_child_weight': [1, 4, 16], # Raise to eliminate leaves with low sample weight
    'subsample': [.7], # Each boosting round trains on X% of the training data
    'max_depth': [2, 4, 8], # Maximum depth of a single tree
    'reg_lambda': [0, 1], # L2 regularization
}
simple_dict.update(new_simple_dict)

In [8]:
tree = XGBClassifier()
params = settings.update(hypers)
num_round = [2, 4, 8, 16]

tree = tree.fit(X = X_train[feature_set['significant_no_credit']], 
                y = y_train,
                eval_metric='rmse')

y_pred = tree.predict(X_test[feature_set['significant_no_credit']])

print(f"Accuracy: {(y_test == y_pred).sum() / (y_test == y_pred).count()}")

Accuracy: 0.5368421052631579


C:\Users\aidan\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


## Grid Search

In [9]:
estimator = XGBClassifier()

In [10]:
# Configure constant model settings
params = {
    'booster': ['gbtree'], 
    'objective': ['binary:logistic'],
    'eval_metric': ['logloss'], # TODO: Suppress the warning down there. 
    'nthread': [multiprocessing.cpu_count()],
    'eta': [0.01, .10, .25, .50], # learning rate, reduces size of error adjustment
    'gamma': [0, 0.5, 5], # Raise to reduce overfitting (prevents low-magnitude leaves)    
    'min_child_weight': [1, 4, 16], # Raise to eliminate leaves with low sample weight
    'subsample': [.7, 1], # Each boosting round trains on X% of the training data
    'max_depth': [2, 4, 8], # Maximum depth of a single tree
    'reg_lambda': [0, 1], # L2 regularization
}

In [11]:
grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=params,
    scoring = ['neg_brier_score', 'accuracy', 'balanced_accuracy', 'precision', 'recall', 'roc_auc'],
    n_jobs = multiprocessing.cpu_count(),
    cv = 5,
    verbose=10,
    refit = 'neg_brier_score'
)

In [12]:
# For each potential set of features
for name, features in feature_set.items():
    
    # Fit each classifier in the grid search
    grid_search.fit(X_train[features], y_train)
    
    # Save performance with dynamic naming
    performance = pd.DataFrame(grid_search.cv_results_)
    performance.to_pickle(f'output_data/04_{name}_performance_0001.pkl')

Fitting 5 folds for each of 432 candidates, totalling 2160 fits


C:\Users\aidan\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


## Second Grid Search

After a round of performance evaluation, I've picked some hyperparameters and come back to test more. Thanks to the scoring of our earlier models, we know our tuning is improving model performance on the validation set. 

Here are the hyperparameters we'd like to continue improving upon:

* Maximum Depth of a Tree
* Minimum Weight of a Leaf
* Gamma Parameter
* Eta Parameter

In [13]:
params_0002 = {
    'booster': ['gbtree'], 
    'objective': ['binary:logistic'],
    'eval_metric': ['logloss'], # TODO: Suppress the warning down there. 
    'nthread': [multiprocessing.cpu_count()],
    'eta': [.05, .01, .001, .0001], 
    'gamma': [1, 10, 50, 100], # Raise to reduce overfitting (prevents low-magnitude leaves)    
    'min_child_weight': [8, 24, 48], # Raise to eliminate leaves with low sample weight
    'max_depth': [4, 8, 16, 32], # Maximum depth of a single tree
}

In [14]:
grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=params_0002,
    scoring = ['neg_brier_score', 'accuracy', 'balanced_accuracy', 'precision', 'recall', 'roc_auc'],
    n_jobs = multiprocessing.cpu_count(),
    cv = 5,
    verbose=10,
    refit = 'neg_brier_score'
)

In [15]:
# For each potential set of features
for name, features in feature_set.items():
    
    # Fit each classifier in the grid search
    grid_search.fit(X_train[features], y_train)
    
    # Save performance with dynamic naming
    performance = pd.DataFrame(grid_search.cv_results_)
    performance.to_pickle(f'output_data/04_{name}_performance_0002.pkl')

Fitting 5 folds for each of 192 candidates, totalling 960 fits


C:\Users\aidan\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


## Third Grid Search

In [16]:
# Configure constant model settings
params_0003 = {
    'booster': ['gbtree'], 
    'objective': ['binary:logistic'],
    'eval_metric': ['logloss'], # TODO: Suppress the warning down there. 
    'nthread': [multiprocessing.cpu_count()],
    'eta': [0.08, .12, .0005, .4], # learning rate, reduces size of error adjustment
    'gamma': [0.25, .75, 2, 8], # Raise to reduce overfitting (prevents low-magnitude leaves)    
    'min_child_weight': [8, 12, 20], # Raise to eliminate leaves with low sample weight
    'subsample': [.7, 1], # Each boosting round trains on X% of the training data
    'max_depth': [2, 4, 8], # Maximum depth of a single tree
    'reg_lambda': [0, 0.5, 1, 1.5], # L2 regularization
}

In [17]:
grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=params_0003,
    scoring = ['neg_brier_score', 'accuracy', 'balanced_accuracy', 'precision', 'recall', 'roc_auc'],
    n_jobs = multiprocessing.cpu_count(),
    cv = 5,
    verbose=10,
    refit = 'neg_brier_score'
)

In [ ]:
# For each potential set of features
for name, features in feature_set.items():
    
    # Fit each classifier in the grid search
    grid_search.fit(X_train[features], y_train)
    
    # Save performance with dynamic naming
    performance = pd.DataFrame(grid_search.cv_results_)
    performance.to_pickle(f'output_data/04_{name}_performance_0003.pkl')

Fitting 5 folds for each of 1152 candidates, totalling 5760 fits


## Show ROC Curve

Using a previously estimated model saved as a pickle. 

In [12]:
best_estimator_0003 = pickle.load(open("output_data/04_best_estimator_0003.pkl", "rb"))

In [14]:
RocCurveDisplay.from_estimator(best_estimator_0003, X_test[feature_set['significant_no_credit']], y_test)

ValueError: Feature shape mismatch, expected: 63, got 11

# Export Results

In [ ]:
# Export data
X_train.to_pickle('output_data/04_X_train.pkl')
X_test.to_pickle('output_data/04_X_test.pkl')

y_train.to_pickle('output_data/04_y_train.pkl')
y_test.to_pickle('output_data/04_y_test.pkl')

In [ ]:
# Export best estimator
best_estimator = grid_search.best_estimator_
pickle.dump(best_estimator, open("output_data/04_best_estimator_0003.pkl", "wb"))

## Future Improvements

Gradient boosting improvements. XGBoost allows you to place weights on the importance of each data point. Bad loans are more important than good, and some data points are harder to classify than others. Give those data points more weight. 